# Tertiaire

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df_tertiaire = pd.read_csv('notebooks/rapprochements/DPE/dpe-v2-tertiaire-2.csv', sep=',', usecols=['N°DPE', 'Identifiant__BAN', 'Code_postal_(BAN)', 'Code_postal_(brut)'])
df_tertiaire['file'] = 'tertiaire'

# df_neuf = pd.read_csv('notebooks/rapprochements/DPE/.csv', sep=',', usecols=['N°DPE', 'Identifiant__BAN', 'Code_postal_(BAN)', 'Code_postal_(brut)'])
# df_neuf['file'] = 'logement-neufs'


# df_tertiaire.head()

df = pd.concat([df_tertiaire])

In [6]:
df.shape

(323878, 5)

In [7]:
df.head()

,N°DPE,Identifiant__BAN,Code_postal_(BAN),Code_postal_(brut),file
0,2363T1680837R,63124_0035_00024_ter,63800.0,63800,tertiaire
1,2269T2615953W,69228_0086_00707,69440.0,69440,tertiaire
2,2357T4209637Z,57540_0300_00004,57370.0,57370,tertiaire
3,2360T0127880L,60073_0075_00007,60650.0,60650,tertiaire
4,2271T0492563C,71309_0070_00014,71710.0,71710,tertiaire


In [11]:
# nombres de lignes qui n'ont pas d'identifiant BAN
df['Identifiant__BAN'].isnull().sum()

np.int64(838)

In [7]:
from batid.models import Building
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

def get_buildings_from_ban_id(ban_id):
    buildings = Building.objects.filter(addresses_read_only__id=ban_id).values('rnb_id')
    return [b['rnb_id'] for b in buildings]

In [8]:
# df_sample = df_select.sample(10)

df_rapprochement = df_select.copy()
df_rapprochement['rnb_id_from_ban_id'] = df_rapprochement['Identifiant__BAN'].apply(get_buildings_from_ban_id)

In [9]:
df_rapprochement.to_csv('notebooks/rapprochements/DPE/dpe-v2-logements-neufs-rapprochement.csv', index=False)

In [15]:
# load the result data from file
df_results = pd.read_csv('notebooks/rapprochements/DPE/dpe-v2-logements-neufs-rapprochement.csv', sep=',')
df_results.head()

,N°DPE,Date_réception_DPE,Identifiant__BAN,Score_BAN,Statut_géocodage,rnb_id_from_ban_id
0,2133N0419762I,2021-09-30,33200_0754_00042,0.80,adresse non géocodée ban car aucune correspond...,"['MS83BSGXRQRS', 'C6H83GRM1VNZ', 'C8BCT1PJ22FE']"
1,2262N1677399D,2022-07-22,62360_0125,0.72,adresse non géocodée ban car aucune correspond...,[]
2,2333N0960129T,2023-03-22,33077_0018,0.37,adresse géocodée ban à l'adresse,[]
3,2428N1607113P,2024-05-03,28137_0047,0.40,adresse géocodée ban à l'adresse,[]
4,2317N4122363N,2023-12-01,17306_0076,0.71,adresse non géocodée ban car aucune correspond...,[]


In [16]:
# split the df dataframe into 100 sub data frames
from concurrent.futures import ThreadPoolExecutor

dfs = [df_results.iloc[i*df_results.shape[0]//100:(i+1) * df_results.shape[0]//100] for i in range(100)]

# create a function to remove small surfaces
def remove_small_surfaces(rnb_ids):
    results = []
    for rnb_id in rnb_ids:
        raw_sql = f"SELECT id, ST_AREA(shape::geography) FROM batid_building WHERE rnb_id = '{rnb_id}'"
        total_surface = Building.objects.raw(raw_sql)[0].st_area
        if total_surface > 25:
            results.append(rnb_id)
    return results

# multi thread apply the function to the dataframes and concatenate the results
def rnb_id_filtered(df):
    df_copy = df.copy()
    df_copy['rnb_id_filtered'] = df['rnb_id_from_ban_id'].apply(eval).apply(remove_small_surfaces)
    return df_copy

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(rnb_id_filtered, dfs)
    df_filtered = pd.concat(results)
    df_filtered.to_csv('notebooks/rapprochements/DPE/dpe-v2-logements-neufs-rapprochement-filtered.csv', index=False)

In [17]:
df_results_filtered = pd.read_csv('notebooks/rapprochements/DPE/dpe-v2-logements-neufs-rapprochement-filtered.csv', sep=',')

Pourcentage de match 1-1 avant filtre des petites surfaces

In [18]:
df_results_filtered['rnb_id_from_ban_id'].apply(eval).apply(len).value_counts()[1] / df_results.shape[0] * 100

25.76109659941749

Pourcentage de match 1-1 APRES filtre des petites surfaces

In [19]:
df_results_filtered['rnb_id_filtered'].apply(eval).apply(len).value_counts()[1] / df_results.shape[0] * 100

26.11196188943866

Ecriture de rapprochement fichier final pour envoi 

In [21]:
df_results_filtered[['N°DPE', 'Date_réception_DPE', 'rnb_id_filtered']].to_csv('notebooks/rapprochements/DPE/dpe-v2-logements-neufs-rapprochement-final-results.csv', index=False, sep=',')